In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./sampled_dataset_reduced.csv')

In [3]:
# Find total missing values for DISCOVERY_TIME
discovery_time_df = df[['DISCOVERY_TIME']]
total = discovery_time_df.isnull().sum().sort_values(ascending = False)
percent = (discovery_time_df.isnull().sum()/discovery_time_df.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys = ['Total missing values', 'Percentage of missing values'])
missing_data

,Total missing values,Percentage of missing values
DISCOVERY_TIME,176408,0.469044


In [4]:
# Replace missing values for discovery time by average of all the time values on that discovery_date
df['DISCOVERY_TIME_NEW'] = df['DISCOVERY_TIME']
df['DISCOVERY_TIME_NEW'].fillna(df.groupby(['DISCOVERY_DATE'])['DISCOVERY_TIME'].transform('mean'), inplace=True)

In [5]:
# Find total missing values again
discovery_time_df = df[['DISCOVERY_TIME_NEW']]
total = discovery_time_df.isnull().sum().sort_values(ascending = False)
percent = (discovery_time_df.isnull().sum()/discovery_time_df.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys = ['Total missing values', 'Percentage of missing values'])
missing_data

,Total missing values,Percentage of missing values
DISCOVERY_TIME_NEW,622,0.001654


As we see, 622 values are still missing. This is because for some groups of DISCOVERY_DATE, all the values for DISCOVERY_TIME are missing. For such data points, replace the missing discovery_time by the mean of the samples

In [6]:
from scipy import stats
most_freq_time, count = stats.mode(df['DISCOVERY_TIME_NEW'])
disc_time_mode = int(most_freq_time)
df['DISCOVERY_TIME_NEW'].replace(np.nan, disc_time_mode, inplace=True)

In [7]:
# Find total missing values again
discovery_time_df = df[['DISCOVERY_TIME_NEW']]
total = discovery_time_df.isnull().sum().sort_values(ascending = False)
percent = (discovery_time_df.isnull().sum()/discovery_time_df.isnull().count()).sort_values(ascending = False)
missing_data = pd.concat([total, percent], axis=1, keys = ['Total missing values', 'Percentage of missing values'])
missing_data

,Total missing values,Percentage of missing values
DISCOVERY_TIME_NEW,0,0.0


All missing values are not imputed. Delete the temporary column and replace values in original column

In [8]:
df['DISCOVERY_TIME'] = df['DISCOVERY_TIME_NEW']
_ = df.drop(columns=['DISCOVERY_TIME_NEW'], axis=1, inplace=True)
df['DISCOVERY_TIME'] = df['DISCOVERY_TIME'].apply(np.int64)

In [9]:
# Save the transformed dataset to csv
df.to_csv('missing_values_discovery_time_dataset.csv', index= False)